# `GridDataset`

Nata represents grids through the `GridDataset` object. The next section discusses the API of `GridDataset`, how to create `GridDataset`, and the basic properties of those objects.

`GridDataset` can be imported from the containers module of nata

In [1]:
from nata.containers import GridDataset

## Creating `GridDatasets`

### Using array-like objects

Creating a `GridDataset` container from an array-like object can be done through the class method `GridDataset.from_array`.

In [3]:
grid = GridDataset.from_array([1, 2, 3, 4])
grid

GridDataset(name='unnamed', label='unnamed', unit='', shape=(4,), iteration=0, time=0.0, grid_axes=[Axis('axis0', len=1, shape=(4,))])

In [3]:
dir(grid)

['__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_function__',
 '__array_ufunc__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '

### General information

As seen in above example, `grid_ds` is a `GridDataset` which stores additional informations about an array-like structure without further specification. It provides information like `name` and `label` which are by default `unnamed`.

In [4]:
print(f"{grid_ds.name  = }")
print(f"{grid_ds.label = }")
print(f"{grid_ds.unit  = }")

NameError: name 'grid_ds' is not defined

These properties can be simple change by setting new values.

In [ ]:
grid_ds.name = "new_name"
grid_ds.label = "new label"
grid_ds.unit = "some unit"

print(f"{grid_ds.name  = }")
print(f"{grid_ds.label = }")
print(f"{grid_ds.unit  = }")


### Array properties 

<div class="alert alert-block alert-info">
Any reference to an array assumes numpy like arrays.
</div>

Each `GridDataset` acts as it would be an array by providing common array properties. This properties are read-only and can be modified by array functions, e.g., `np.reshape`.

In [ ]:
print(f"{grid_ds.shape = }")
print(f"{grid_ds.ndim  = }")
print(f"{grid_ds.dtype = }")

In addition to the array properties, 

Also an extra information about the unit for `GridDataset` and by default it is empty. However, it behaves similar to a numpy array by providing properties like `.shape`, `.ndim` and `.dtype`.



In general, a `GridDataset` is expected to behave like an array.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(grid_ds)
#plt.plot(grid_ds + 3)
#plt.plot(np.sqrt(grid_ds ** 2.0))

In [ ]:
test = "blub"
("Hallo " "Swen!" " <3" test)

In [ ]:
test = ("Hallo " "Welt")
test

In [ ]:
t1 = True
t2 = False
t3 = True

t1 + t2 + t3

In [ ]:
0 % 7